In [1]:
import numpy as np
import six
import tensorflow as tf
import time
import os

In [6]:
EMBEDDING_DIM = 512

def transform(txt, pad_to=None):
    output = np.asarray([ord(c) for c in txt if ord(c) < 255], dtype=np.int32)
    if pad_to is not None:
        output = output[:pad_to]
        output = np.concatenate([
            np.zeros([pad_to - len(txt)], dtype=np.int32),
            output,
        ])
    return output


def lstm_model(seq_len=100, batch_size=None, stateful=True):
#   """Language model: predict the next word given the current word."""
    source = tf.keras.Input(
      name='seed', shape=(seq_len,), batch_size=batch_size, dtype=tf.int32)

    embedding = tf.keras.layers.Embedding(input_dim=256, output_dim=EMBEDDING_DIM)(source)
    lstm_1 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(embedding)
    lstm_2 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(lstm_1)
    predicted_char = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='softmax'))(lstm_2)
    model = tf.keras.Model(inputs=[source], outputs=[predicted_char])
    model.compile(
      optimizer=tf.train.RMSPropOptimizer(learning_rate=0.01),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
    return model

In [ ]:
BATCH_SIZE = 1
PREDICT_LEN = 20000

# Keras requires the batch size be specified ahead of time for stateful models.
# We use a sequence length of 1, as we will be feeding in one character at a 
# time and predicting the next character.
prediction_model = lstm_model(seq_len=1, batch_size=BATCH_SIZE, stateful=True)
prediction_model.load_weights('model.h5')

# We seed the model with our initial string, copied BATCH_SIZE times

seed_txt = 'Mr. Speaker, I congratulate my colleague, the member for Richmond—Arthabaska, on his excellent speech. '
seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)

# First, run the seed forward to prime the state of the model.
prediction_model.reset_states()
for i in range(len(seed_txt) - 1):
    prediction_model.predict(seed[:, i:i + 1])

# Now we can accumulate predictions!
predictions = [seed[:, -1:]]
for i in range(PREDICT_LEN):
    last_word = predictions[-1]
    next_probits = prediction_model.predict(last_word)[:, 0, :]

    # sample from our output distribution
    next_idx = [
      np.random.choice(256, p=next_probits[i])
      for i in range(BATCH_SIZE)
    ]
    predictions.append(np.asarray(next_idx, dtype=np.int32))

for i in range(BATCH_SIZE):
    p = [predictions[j][i] for j in range(PREDICT_LEN)]
    generated = ''.join([chr(c) for c in p])
    assert len(generated) == PREDICT_LEN, 'Generated text too short'

In [30]:
for x in generated.split('\r\n'):
    if x[-1] != '.':
        print(x[:x.rfind('.') + 1])
    else:
        print(x)
    print('\n -- \n')

 Ottawa be allowed to see the path that even participate in infrastructure and the people with great improvement next year.I would like to take the opportunity to move forward with parallell business apparent trading and developed Minister of Aboriginal Affairs and my hon. colleague has voted against unique mental headline and throwing for a prime minister, both the end to our children. A few minutes of Refugee Labrand is proposed supervision.

 -- 

Liberal: Mr. Speaker, the minister has introduced rights. He is a cut of life. What the company's army, sliper price on a coalition that it is entirely on our relationship, which was the third-time for NanaimoLadyseldbaugoBries to vetill the environment, the women seek it to the chain, like over his constituents need to see the comments the ucceAssist Government will have done.  My view is an important nothing or quite with the government's beliefs and, once again. In fact, last year, I want to thank any government has the picture with her